In [76]:
import requests
import os
from bs4 import BeautifulSoup

apikey = os.environ['TRANSLINK_API']

############# DO NOT DO THIS ################
# store your API key(s) in environment variables
apikey


KeyError: 'TRANSLINK_AP'

In [34]:
# pip/conda 'magic' commands
# %pip install bs4
# %conda install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


**Documentation:** <br>
https://www.translink.ca/about-us/doing-business-with-translink/app-developer-resources/rtti

## Basic Response from API

In [13]:
URL = 'http://api.translink.ca/rttiapi/v1/status' + '?apikey=' + apikey
response = requests.get(URL)
response.text


'<Statuses xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><Status><Name>Schedule</Name><Value>Online</Value></Status><Status><Name>Location</Name><Value>Online</Value></Status></Statuses>'

### Parsing XML

`pip install lxml` <br>
`conda install lxml`


In [ ]:
# %conda install lxml

In [18]:
soup = BeautifulSoup(response.text, 'lxml')
print(soup.prettify())
# soup

<html>
 <body>
  <statuses xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
   <status>
    <name>
     Schedule
    </name>
    <value>
     Online
    </value>
   </status>
   <status>
    <name>
     Location
    </name>
    <value>
     Online
    </value>
   </status>
  </statuses>
 </body>
</html>


In [36]:
# Example 1: get information about a specific bus stop
request_url = 'http://api.translink.ca/rttiapi/v1/stops/61935?apikey=' + apikey

response = requests.get(request_url)
response.text

'<Stop xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><StopNo>61935</StopNo><Name>UBC EXCHANGE BAY 7</Name><BayNo>4</BayNo><City>VANCOUVER</City><OnStreet>UBC EXCHANGE</OnStreet><AtStreet>BAY 7</AtStreet><Latitude>49.267419</Latitude><Longitude>-123.246831</Longitude><WheelchairAccess>1</WheelchairAccess><Distance>-1</Distance><Routes>099</Routes></Stop>'

In [23]:
soup = BeautifulSoup(response.text, 'lxml')
soup

<html><body><stop xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><stopno>61935</stopno><name>UBC EXCHANGE BAY 7</name><bayno>4</bayno><city>VANCOUVER</city><onstreet>UBC EXCHANGE</onstreet><atstreet>BAY 7</atstreet><latitude>49.267419</latitude><longitude>-123.246831</longitude><wheelchairaccess>1</wheelchairaccess><distance>-1</distance><routes>099</routes></stop></body></html>

In [28]:
# extract the stop number
stop_number = soup.find('stopno').text
stop_number

'61935'

In [32]:
# extract the city
city = soup.find('city').text
city.capitalize()

'Vancouver'

In [31]:
# extract on street
on_street = soup.find('onstreet')
on_street

<onstreet>UBC EXCHANGE</onstreet>

In [30]:
soup.find('stop')

<stop xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><stopno>61935</stopno><name>UBC EXCHANGE BAY 7</name><bayno>4</bayno><city>VANCOUVER</city><onstreet>UBC EXCHANGE</onstreet><atstreet>BAY 7</atstreet><latitude>49.267419</latitude><longitude>-123.246831</longitude><wheelchairaccess>1</wheelchairaccess><distance>-1</distance><routes>099</routes></stop>

### Parsing JSON

In [53]:
# Example 2: get estimates of bus schedules at a specific stop
request_url = 'http://api.translink.ca/rttiapi/v1/stops/' + '61935' + '/estimates?apikey=' + apikey
header_dict = {'accept': 'application/JSON'}
response = requests.get(request_url, headers=header_dict)
response.text
res_json = response.json()
res_json[0]


{'RouteNo': '099',
 'RouteName': 'COMMERCIAL-BROADWAY/UBC (B-LINE)',
 'Direction': 'EAST',
 'RouteMap': {'Href': 'https://nb.translink.ca/geodata/099.kmz'},
 'Schedules': [{'Pattern': 'E1',
   'Destination': "COMM'L-BDWAY STN",
   'ExpectedLeaveTime': '11:50am',
   'ExpectedCountdown': 3,
   'ScheduleStatus': '*',
   'CancelledTrip': False,
   'CancelledStop': False,
   'AddedTrip': False,
   'AddedStop': False,
   'LastUpdate': '10:50:04 am'},
  {'Pattern': 'E1',
   'Destination': "COMM'L-BDWAY STN",
   'ExpectedLeaveTime': '11:57am',
   'ExpectedCountdown': 10,
   'ScheduleStatus': '*',
   'CancelledTrip': False,
   'CancelledStop': False,
   'AddedTrip': False,
   'AddedStop': False,
   'LastUpdate': '10:57:05 am'},
  {'Pattern': 'E1',
   'Destination': "COMM'L-BDWAY STN",
   'ExpectedLeaveTime': '12:05pm',
   'ExpectedCountdown': 18,
   'ScheduleStatus': '*',
   'CancelledTrip': False,
   'CancelledStop': False,
   'AddedTrip': False,
   'AddedStop': False,
   'LastUpdate': '11:05:

In [55]:
for schedule in res_json[0]['Schedules']:
    print('Leaving at ' + schedule['ExpectedLeaveTime'])

Leaving at 11:50am
Leaving at 11:57am
Leaving at 12:05pm
Leaving at 12:12pm
Leaving at 12:20pm
Leaving at 12:27pm


In [57]:
# Example 3: get all bus stops near a (latitude, longitude) coordinate (more parameters)
request_url = 'https://api.translink.ca/rttiapi/v1/stops?apikey=' + apikey + '&lat=' + '49.18' + '&long=' + '-122.85' + '&radius=1000'
response = requests.get(request_url, headers={'accept': 'application/JSON'})
response.json()

[{'StopNo': 54997,
  'Name': 'SB KING GEORGE BLVD FS 98 AVE',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': 'KING GEORGE BLVD',
  'AtStreet': '98 AVE',
  'Latitude': 49.179601,
  'Longitude': -122.845814,
  'WheelchairAccess': 1,
  'Distance': 307,
  'Routes': '314, 321, 329'},
 {'StopNo': 54986,
  'Name': 'EB 96 AVE FS 134 ST',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': '96 AVE',
  'AtStreet': '134 ST',
  'Latitude': 49.176992,
  'Longitude': -122.850709,
  'WheelchairAccess': 0,
  'Distance': 338,
  'Routes': '314, 329'},
 {'StopNo': 54999,
  'Name': 'WB 96 AVE FS 134 ST',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': '96 AVE',
  'AtStreet': '134 ST',
  'Latitude': 49.177159,
  'Longitude': -122.851794,
  'WheelchairAccess': 0,
  'Distance': 342,
  'Routes': '314, 329'},
 {'StopNo': 54989,
  'Name': 'NB KING GEORGE BLVD FS FRASER HWY',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': 'KING GEORGE BLVD',
  'AtStreet': 'FRASER HWY',
  'Latitude': 49.181116,
  'Longitude

## Using the requests.get() 'params' parameter

In [68]:
URL = 'https://api.translink.ca/rttiapi/v1/stops'
params_dict = { 
             'lat' : '49.18',
             'long' : '-122.85',
             'radius' : '1000',
             'apikey' : apikey
              }
headers_dict = {'accept': 'application/JSON'}
response = requests.get(URL, params=params_dict, headers=headers_dict)
res_json = response.json()
res_json


[{'StopNo': 54997,
  'Name': 'SB KING GEORGE BLVD FS 98 AVE',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': 'KING GEORGE BLVD',
  'AtStreet': '98 AVE',
  'Latitude': 49.179601,
  'Longitude': -122.845814,
  'WheelchairAccess': 1,
  'Distance': 307,
  'Routes': '314, 321, 329'},
 {'StopNo': 54986,
  'Name': 'EB 96 AVE FS 134 ST',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': '96 AVE',
  'AtStreet': '134 ST',
  'Latitude': 49.176992,
  'Longitude': -122.850709,
  'WheelchairAccess': 0,
  'Distance': 338,
  'Routes': '314, 329'},
 {'StopNo': 54999,
  'Name': 'WB 96 AVE FS 134 ST',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': '96 AVE',
  'AtStreet': '134 ST',
  'Latitude': 49.177159,
  'Longitude': -122.851794,
  'WheelchairAccess': 0,
  'Distance': 342,
  'Routes': '314, 329'},
 {'StopNo': 54989,
  'Name': 'NB KING GEORGE BLVD FS FRASER HWY',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': 'KING GEORGE BLVD',
  'AtStreet': 'FRASER HWY',
  'Latitude': 49.181116,
  'Longitude

In [62]:
res_json[0]

{'StopNo': 54997,
 'Name': 'SB KING GEORGE BLVD FS 98 AVE',
 'BayNo': 'N',
 'City': 'SURREY',
 'OnStreet': 'KING GEORGE BLVD',
 'AtStreet': '98 AVE',
 'Latitude': 49.179601,
 'Longitude': -122.845814,
 'WheelchairAccess': 1,
 'Distance': 307,
 'Routes': '314, 321, 329'}

In [63]:
# for index in range(len(response)):
#     print(response[index]['WheelchairAccess'])

for stop in res_json:
    print(stop['WheelchairAccess'])

1
0
0
1
1
1
1
1
0
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [64]:
wheelchair_access = {}
for stop in res_json:
    access = stop['WheelchairAccess']
    name = stop['Name']
    if access == 1:
        wheelchair_access[name] = True
#         print(name, 'Yes')
    else:
        wheelchair_access[name] = False
#         print(name, 'No')
        
wheelchair_access

{'SB KING GEORGE BLVD FS 98 AVE': True,
 'EB 96 AVE FS 134 ST': False,
 'WB 96 AVE FS 134 ST': False,
 'NB KING GEORGE BLVD FS FRASER HWY': True,
 'WB 96 AVE FS KING GEORGE BLVD': True,
 'NB KING GEORGE BLVD FS 96 AVE': True,
 'KING GEORGE STN BAY 4': True,
 'KING GEORGE STN BAY 1': True,
 'KING GEORGE STATION PLATFORM 2': False,
 'NB 132 ST NS 98 AVE': True,
 'SB 132 ST FS 98 AVE': False,
 'KING GEORGE STN BAY 2': True,
 'KING GEORGE STATION PLATFORM 1': False,
 'EB OLD YALE RD AT 13500 BLOCK': True,
 'KING GEORGE STN BAY 5': True,
 'SB KING GEORGE BLVD FS 96 AVE': True,
 'EB 96 AVE FS 132 ST': True,
 'NB 132 ST FS 96 AVE': True,
 'WB FRASER HWY FS WHALLEY BLVD': True,
 'KING GEORGE STN BAY 3': True,
 'WB 96 AVE FS 132 ST': True,
 'NB KING GEORGE BLVD FS 94A AVE': True,
 'NB KING GEORGE BLVD FS 100 AVE': True,
 'SB 132 ST FS 96 AVE': True,
 'EB 96 AVE FS 137 ST': True,
 'EB OLD YALE RD FS UNIVERSITY DR': True,
 'SB 132 ST FS 100 AVE': True,
 'NB 132 ST FS 100 AVE': True,
 'WB OLD YALE

## Now with XML

In [69]:
URL = 'https://api.translink.ca/rttiapi/v1/stops'
    
params_dict = { 
             'lat' : '49.18',
             'long' : '-122.85',
             'radius' : '600',
             'apikey' : apikey
              }
response = requests.get(URL, params=params_dict)
response.text

'<Stops xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><Stop><StopNo>54997</StopNo><Name>SB KING GEORGE BLVD FS 98 AVE</Name><BayNo>N</BayNo><City>SURREY</City><OnStreet>KING GEORGE BLVD</OnStreet><AtStreet>98 AVE</AtStreet><Latitude>49.179601</Latitude><Longitude>-122.845814</Longitude><WheelchairAccess>1</WheelchairAccess><Distance>307</Distance><Routes>314, 321, 329</Routes></Stop><Stop><StopNo>54986</StopNo><Name>EB 96 AVE FS 134 ST</Name><BayNo>N</BayNo><City>SURREY</City><OnStreet>96 AVE</OnStreet><AtStreet>134 ST</AtStreet><Latitude>49.176992</Latitude><Longitude>-122.850709</Longitude><WheelchairAccess>0</WheelchairAccess><Distance>338</Distance><Routes>314, 329</Routes></Stop><Stop><StopNo>54999</StopNo><Name>WB 96 AVE FS 134 ST</Name><BayNo>N</BayNo><City>SURREY</City><OnStreet>96 AVE</OnStreet><AtStreet>134 ST</AtStreet><Latitude>49.177159</Latitude><Longitude>-122.851794</Longitude><WheelchairAccess>0</WheelchairAccess><Distance>342</Distance><Routes>314, 329</Routes><

In [70]:
soup = BeautifulSoup(response.text, 'lxml')
print(soup.prettify())

<html>
 <body>
  <stops xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
   <stop>
    <stopno>
     54997
    </stopno>
    <name>
     SB KING GEORGE BLVD FS 98 AVE
    </name>
    <bayno>
     N
    </bayno>
    <city>
     SURREY
    </city>
    <onstreet>
     KING GEORGE BLVD
    </onstreet>
    <atstreet>
     98 AVE
    </atstreet>
    <latitude>
     49.179601
    </latitude>
    <longitude>
     -122.845814
    </longitude>
    <wheelchairaccess>
     1
    </wheelchairaccess>
    <distance>
     307
    </distance>
    <routes>
     314, 321, 329
    </routes>
   </stop>
   <stop>
    <stopno>
     54986
    </stopno>
    <name>
     EB 96 AVE FS 134 ST
    </name>
    <bayno>
     N
    </bayno>
    <city>
     SURREY
    </city>
    <onstreet>
     96 AVE
    </onstreet>
    <atstreet>
     134 ST
    </atstreet>
    <latitude>
     49.176992
    </latitude>
    <longitude>
     -122.850709
    </longitude>
    <wheelchairaccess>
     0
    </wheelchairaccess>
    <dis

In [72]:
stops = soup.find_all('stop')
stops[5]

<stop><stopno>54988</stopno><name>NB KING GEORGE BLVD FS 96 AVE</name><bayno>N</bayno><city>SURREY</city><onstreet>KING GEORGE BLVD</onstreet><atstreet>96 AVE</atstreet><latitude>49.177621</latitude><longitude>-122.845422</longitude><wheelchairaccess>1</wheelchairaccess><distance>425</distance><routes>314, 321, 326, 329, 394, R1</routes></stop>

In [73]:
for stop in stops:
    print(stop.find('wheelchairaccess').text)

1
0
0
1
1
1
1
1
0
1
0
1
0
1
1
1
1
1
1
1


In [74]:
wheelchair_access = {}
for stop in stops:
    access = stop.find('wheelchairaccess').text
    # use element.text attribute
    name = stop.find('name').text
    if access == 1:
        wheelchair_access[name] = True
#         print(name, 'Yes')
    else:
        wheelchair_access[name] = False
#         print(name, 'No')
        
wheelchair_access

{'SB KING GEORGE BLVD FS 98 AVE': False,
 'EB 96 AVE FS 134 ST': False,
 'WB 96 AVE FS 134 ST': False,
 'NB KING GEORGE BLVD FS FRASER HWY': False,
 'WB 96 AVE FS KING GEORGE BLVD': False,
 'NB KING GEORGE BLVD FS 96 AVE': False,
 'KING GEORGE STN BAY 4': False,
 'KING GEORGE STN BAY 1': False,
 'KING GEORGE STATION PLATFORM 2': False,
 'NB 132 ST NS 98 AVE': False,
 'SB 132 ST FS 98 AVE': False,
 'KING GEORGE STN BAY 2': False,
 'KING GEORGE STATION PLATFORM 1': False,
 'EB OLD YALE RD AT 13500 BLOCK': False,
 'KING GEORGE STN BAY 5': False,
 'SB KING GEORGE BLVD FS 96 AVE': False,
 'EB 96 AVE FS 132 ST': False,
 'NB 132 ST FS 96 AVE': False,
 'WB FRASER HWY FS WHALLEY BLVD': False}